In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
def read_url(url):
    response = requests.get(url)
    page=response.text
    soup=BeautifulSoup(page,'lxml')
    return soup

In [3]:
#load moves url based on novel

url_list=[]

for i in range(1,70):
    url_list.append('https://www.imdb.com/search/keyword/?keywords=based-on-novel&mode=advanced&&page={}&ref_=kw_ref_key&sort=moviemeter,asc'.format(i))
                     

In [4]:
i_scores=[]

for url in url_list:
    scores=read_url(url).find_all("div", {"class": "lister-item-content"})
    for p in scores:
        try:
            if p.find('div')['class']==['ratings-bar']:
                i_scores.append(float(p.find('div',{'class':'ratings-bar'}).find('strong').get_text()))
        except:
                i_scores.append(np.nan)

In [5]:
runtimes=[]
for url in url_list:
    RT=read_url(url).find_all("p", {"class": "text-muted"})
    for p in RT:
        if p.find('span')==None:
            pass
        elif p.find_all('span')[0]['class']==['certificate'] and p.find_all('span')[2]['class']!=['genre']:
            runtimes.append(str(p.find_all('span')[2].get_text())) 
            
        elif p.find_all('span')[0]['class']==['runtime']:
             runtimes.append(str(p.find_all('span')[0].get_text()))
        else:    
            runtimes.append(np.nan)

In [6]:
ratings=[]

for url in url_list:
    rate=read_url(url).find_all("p", {"class": "text-muted"})
    for p in rate:
        if p.find('span')==None:
            pass
        elif p.find('span')['class']==['certificate']:
            ratings.append(str(p.find('span',{'class':'certificate'}).get_text()))
        else:
            ratings.append(np.nan)

In [7]:
genres=[]
for url in url_list:
    genre=read_url(url).find_all('span',{"class":"genre"})
    for item in [g.get_text() for g in genre]:
        genres.append(str(item).replace('\n','').strip())

In [8]:
gset=['Action','Adventure','Biography','Comedy','Crime','Documentary','Drama','Family','History','Horror',
      'Music','Mystery','Romance','Sport','Thriller','War','Western']
Action,Adventure,Biography,Comedy,Crime,Documentary,Drama,Family,History,Horror,Music,Mystery,Romance,Sport,Thriller,War,Western=([] for i in range(len(gset)))

glist=[Action,Adventure,Biography,Comedy,Crime,Documentary,Drama,Family,History,Horror,Music,Mystery,Romance,Sport,Thriller,War,Western]
for g in genres:
    for c in gset:
        if c in g:
            glist[gset.index(c)].append(1)
        else:
            glist[gset.index(c)].append(0)

In [10]:
titles=[]
for url in url_list:
    d1=read_url(url).find_all('h3',{"class":"lister-item-header"})
    for h in d1:
        for link in h.find_all('a'):
            titles.append(str(link.get_text().strip()))

In [11]:
imdb_movies=pd.DataFrame({"Title":titles,'Rating':ratings,'Runtime':runtimes,'IMDB_Score':i_scores, 'Action':Action,'Adventure':Adventure,'Biography':Biography,'Comedy':Comedy,'Crime':Crime,'Documentary':Documentary,'Drama':Drama,'Family':Family,'History':History,'Horror':Horror,'Music':Music,'Mystery':Mystery,'Romance':Romance,'Sport':Sport,'Thriller':Thriller,'War':War,'Western':Western})
imdb_movies=imdb_movies[['Title','Rating','Runtime','IMDB_Score','Action','Adventure','Biography','Comedy','Crime','Documentary','Drama','Family','History','Horror','Music','Mystery','Romance','Sport','Thriller','War','Western']]

#Replace movie rating with value 1-4 based on typical box office performance
imdb_movies['Rating']=imdb_movies['Rating'].replace('G',1).replace('PG',2).replace('PG-13',4).replace('R',3).replace([i for i in imdb_movies['Rating'] if i not in ['G','PG','PG-13','R']],np.nan)


In [12]:
#imdb_movies

In [13]:
imdb_movies.to_csv('imdb_movies.csv')

In [14]:
#https://www.boxofficemojo.com/chart/top_opening_day/?ref_=bo_csd_ac

In [15]:
def read_url_3(url):
    response = requests.get(url)
    page=response.text
    soup=BeautifulSoup(page,'lxml')
    return soup

In [16]:
#load moves url based on novel

url_list3=[]
o=[200,400,600,800]
url_list3.append('https://www.boxofficemojo.com/chart/top_opening_day/')
for i in o:
    url_list3.append('https://www.boxofficemojo.com/chart/top_opening_day/?offset='+ str(i))
   

In [17]:
#https://www.boxofficemojo.com/year/world/2018/

In [18]:
Theaters=[]
titles=[]
tot_gross=[]
for url in url_list3:
    
    table1=read_url_3(url).find_all('table')[0]
    title=table1.find_all('td')[1::9]
    Theater=table1.find_all('td')[5::9]
    tot_gro=table1.find_all('td')[3::9]
    
    
    for i in title:
        titles.append(str(i.get_text()))
  
    for t in Theater:
        cleaned = str(t.get_text()).replace(',','').replace('-','')
        if not cleaned :
            cleaned = -1
        else:
            cleaned = int(cleaned) 
            Theaters.append(cleaned)
            
    for to in tot_gro:
        tot_gross.append(int(str(to.get_text()).replace('$','').replace(',','')))
        


In [19]:
#titles.index('Demon Slayer: Mugen Train')#627

In [20]:
titles.remove('Demon Slayer: Mugen Train')
tot_gross.remove(tot_gross[627])


In [21]:
mojo_movies=pd.DataFrame({"Title":titles,'Theaters':Theaters,'Total Gross':tot_gross})

mojo_movies

,Title,Theaters,Total Gross
0,Avengers: Endgame,4662,858373000
1,Star Wars: Episode VII - The Force Awakens,4134,936662225
2,Avengers: Infinity War,4474,678815482
3,Star Wars: Episode VIII - The Last Jedi,4232,620181382
4,Harry Potter and the Deathly Hallows: Part 2,4375,381011219
...,...,...,...
994,Burn After Reading,2651,60355347
995,Focus,3323,53862963
996,Holmes & Watson,2719,30573626
997,Smallfoot,4131,83240103


In [22]:
#Collecting data for Title, Budge, and Domestic Gross from The-numbers.com


In [23]:
def read_url_2(url):
    response = requests.get(url)
    page=response.text
    soup=BeautifulSoup(page,'lxml')
    return soup

In [24]:

#https://www.the-numbers.com/movie/budgets/all/


url_list2=[]
p=[101,201,301,401,501,601,701,801,901,1001,1101,1201,1301,1401,1501,1601,1701,1801,1901,2001,2101,2201,2301,2401,2501,2601,2701,2801,2901,3001,3101,3201,3301,3401,3501,3601,3701,3801,3901,4001,4101,4201,4301,4401,4501,4601,4701,4801,4901,5001,5001,5101,5201,5301,5401,5501,5601,5701,5801,5901,6001]
for i in p:
    url_list2.append('https://www.the-numbers.com/movie/budgets/all/'+ str(i))
   

In [25]:

#url='https://www.the-numbers.com/movie/budgets/all/101'
titles=[]
budgets=[]
w_grosses=[]
d_grosses=[]
dates=[]
for url in url_list2:
    
    table1=read_url_2(url).find_all('table')[0]
    m_list=table1.find_all('td')[2::6]
    budget=table1.find_all('td')[3::6]
    w_gross=table1.find_all('td')[5::6]
    d_gross=table1.find_all('td')[4::6]
    date=table1.find_all('td')[1::6]
    #DomesticGross

    for m in m_list:
        titles.append(str(m.find('b').find('a').get_text()))
    for b in budget:
        budgets.append(int(str(b.get_text()).replace('$','').replace(',','')))
    for d in d_gross:
        d_grosses.append(int(str(d.get_text()).replace('$','').replace(',','')))
    for w in w_gross:
        w_grosses.append(int(str(w.get_text()).replace('$','').replace(',','')))
    for d in date:
        cleaned = dates.append(str(d.get_text().replace(',','').strip()))
        #cleaned = dates.append(str(d.get_text().strip()))


    #if not cleaned :
            #dates.append(None)
        #else: 
           # dates.append(datetime.strptime(cleaned,'%b %d %Y'))
       

In [26]:
budget_idx=pd.DataFrame({'Title':titles, 'Budget':budgets,'DomesticGross':d_grosses,'WorldwideGross':w_grosses,'ReleaseDate':dates})
budget_idx

,Title,Budget,DomesticGross,WorldwideGross,ReleaseDate
0,WALL-E,180000000,223808164,532508025,Jun 27 2008
1,Rush Hour 3,180000000,140125968,256585882,Aug 10 2007
2,The Legend of Tarzan,180000000,126643061,348902025,Jun 27 2016
3,Hugo,180000000,73864507,180047784,Nov 23 2011
4,Valerian and the City of a Thousand Planets,180000000,40479370,215098356,Jul 20 2017
...,...,...,...,...,...
6095,Sing,84000,0,0,Unknown
6096,The Foot Fist Way,79000,234286,234286,May 30 2008
6097,Dawn of the Crescent Moon,75000,8799,8799,Jul 17 2015
6098,Queen Crab,75000,0,0,Sep 29 2015


In [27]:
#budget_idx
budget_idx.to_csv('budget_idx.csv')

In [28]:
#MergeMovies from BoxOfficeMojo.com, IMDB.com "Based on novel" movies, and The-numbers.com movies by Title:
movies=pd.merge(imdb_movies,mojo_movies,how='inner',on=['Title'])
movies=pd.merge(movies,budget_idx,how='inner',on=['Title'])
movies.dropna(inplace=True)
movies.drop_duplicates(inplace=True)
movies

,Title,Rating,Runtime,IMDB_Score,Action,Adventure,Biography,Comedy,Crime,Documentary,...,Sport,Thriller,War,Western,Theaters,Total Gross,Budget,DomesticGross,WorldwideGross,ReleaseDate
0,Casino Royale,4.0,144 min,8.0,1,1,0,0,0,0,...,0,1,0,0,3434,167445960,102000000,167365000,594420283,Nov 17 2006
1,Casino Royale,4.0,144 min,8.0,1,1,0,0,0,0,...,0,1,0,0,3434,167445960,12000000,22744718,41744718,Apr 28 1967
4,Dune,2.0,137 min,6.4,1,1,0,0,0,0,...,0,0,0,0,4125,17500000,165000000,40100000,161664741,Sep 15 2021
5,Dune,2.0,137 min,6.4,1,1,0,0,0,0,...,0,0,0,0,4125,17500000,45000000,27447471,27449336,Dec 14 1984
9,It,2.0,192 min,6.8,0,0,0,0,0,0,...,0,0,0,0,4103,327481748,35000000,328828874,701083042,Aug 31 2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,The Taking of Pelham 123,3.0,106 min,6.4,1,0,0,0,1,0,...,0,1,0,0,3074,65452312,110000000,65452312,152364370,Jun 12 2009
185,The Stepford Wives,4.0,93 min,5.3,0,0,0,1,0,0,...,0,0,0,0,3057,59484742,100000000,59475623,96150482,Jun 11 2004
187,Diary of a Wimpy Kid: Rodrick Rules,2.0,100 min,6.5,0,0,0,1,0,0,...,0,0,0,0,3167,52698535,18000000,52698535,73695194,Mar 25 2011
190,Captain Underpants: The First Epic Movie,2.0,89 min,6.2,1,1,0,0,0,0,...,0,0,0,0,3434,73921000,38000000,73921000,126422949,Jun 2 2017


In [31]:
movies.to_csv('movies.csv')

In [32]:
movies

,Title,Rating,Runtime,IMDB_Score,Action,Adventure,Biography,Comedy,Crime,Documentary,...,Sport,Thriller,War,Western,Theaters,Total Gross,Budget,DomesticGross,WorldwideGross,ReleaseDate
0,Casino Royale,4.0,144 min,8.0,1,1,0,0,0,0,...,0,1,0,0,3434,167445960,102000000,167365000,594420283,Nov 17 2006
1,Casino Royale,4.0,144 min,8.0,1,1,0,0,0,0,...,0,1,0,0,3434,167445960,12000000,22744718,41744718,Apr 28 1967
4,Dune,2.0,137 min,6.4,1,1,0,0,0,0,...,0,0,0,0,4125,17500000,165000000,40100000,161664741,Sep 15 2021
5,Dune,2.0,137 min,6.4,1,1,0,0,0,0,...,0,0,0,0,4125,17500000,45000000,27447471,27449336,Dec 14 1984
9,It,2.0,192 min,6.8,0,0,0,0,0,0,...,0,0,0,0,4103,327481748,35000000,328828874,701083042,Aug 31 2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,The Taking of Pelham 123,3.0,106 min,6.4,1,0,0,0,1,0,...,0,1,0,0,3074,65452312,110000000,65452312,152364370,Jun 12 2009
185,The Stepford Wives,4.0,93 min,5.3,0,0,0,1,0,0,...,0,0,0,0,3057,59484742,100000000,59475623,96150482,Jun 11 2004
187,Diary of a Wimpy Kid: Rodrick Rules,2.0,100 min,6.5,0,0,0,1,0,0,...,0,0,0,0,3167,52698535,18000000,52698535,73695194,Mar 25 2011
190,Captain Underpants: The First Epic Movie,2.0,89 min,6.2,1,1,0,0,0,0,...,0,0,0,0,3434,73921000,38000000,73921000,126422949,Jun 2 2017
